# Chapter 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 1.1
### 1.1.a

According to the law of total probability (LOTP) the marginal density of *y* is defined as the weighted average of the conditional densities:
$$p(y) = \sum_{\theta}{p(y|\theta) * p(\theta)}$$
In the present examples with two possible values for theta, this is
$$p(y) = p(y|\theta_1)*p(\theta_1)+p(y|\theta_2)*p(\theta_2)$$
Since both are normal and the thetas a equally likely unconditionally, the equation takes the following form:
$$p(y) = \frac{1}{2}*(\mathcal{N}(y|0, 2^2) + \mathcal{N}(y|1, 2^2))$$

In [2]:
prior = np.array([.5, .5]) # prior on thetas
mus = [0, 1]
x = np.linspace(-4, 4)

## Exercise 9:

## Simulate subjects

In [44]:
def simulate_subs():
    t_max = 420
    subs = []
    t = 0
    while True:
        sub = t + np.random.exponential(10)
        if sub > t_max:
            break
        subs.append(sub)
        t = sub
    n_subs = len(subs)
    return subs, n_subs

## Simulate doctors

In [85]:
def simulate_docs(subs):
    doc_t = np.zeros(3)
    wt = []
    for s in subs:
        duration = np.random.uniform(5, 20)
        if any(doc_t < s):
            wt.append(0)
            doc_t[np.argmin(doc_t)] = s + duration
        else:
            wt.append(doc_t.min() - s)
            doc_t[np.argmin(doc_t)] += duration
    wt_nonzero  = np.array([x for x in wt if not x == 0])
    avg_wait = wt_nonzero.mean() if len(wt_nonzero) > 0 else 0
    n_hadtowait = wt_nonzero.size
    closing_delay = np.max([0, doc_t.max() - 420])
    return n_hadtowait, avg_wait, closing_delay

In [57]:
from tqdm import tqdm
import time

In [92]:
runs = 10000
n_subs, n_hadtowait, avg_wait, closing_delay = [
    np.empty(shape=(runs,), dtype=np.float) for _ in range(4)
]
for r in tqdm(range(runs)):
    subs, n_subs[r] = simulate_subs()
    n_hadtowait[r], avg_wait[r], closing_delay[r] = simulate_docs(subs)

100%|██████████| 10000/10000 [00:07<00:00, 1415.45it/s]


In [93]:
def get_stats(data):
    return [np.percentile(data, x) for x in [25, 50, 75]]

In [94]:
print('Number of subjects: {:.2f}, {:.2f}, {:.2f}'.format(*get_stats(all_n_subs)))
print('Average waiting time: {:.2f}, {:.2f}, {:.2f}'.format(*get_stats(all_avg_wait)))
print('Number of waiting: {:.2f}, {:.2f}, {:.2f}'.format(*get_stats(all_n_hadtowait)))
print('Closing delay {:.2f}, {:.2f}, {:.2f}'.format(*get_stats(all_closing_delay)))

Number of subjects: 38.00, 42.00, 46.00
Average waiting time: 2.54, 3.77, 5.09
Number of waiting: 3.00, 5.00, 8.00
Closing delay 0.00, 5.94, 11.34


## 13

In [6]:
import pandas as pd
from scipy.stats import gamma, poisson

In [4]:
fatal = np.array([24, 25, 31, 31, 22, 21, 26, 20, 16, 22])
deaths = np.array([734, 516, 754, 877, 814, 362, 764, 809, 223, 1066])
deathrate = np.array([19, 12, 15, 16, 14, 6, 13, 13, 3, 15]) / 100
df = pd.DataFrame(
    {
        'fatal': fatal,
        'deaths': deaths,
        'deathrate': deathrate
    },
    index=np.arange(1976, 1986)
)

Using Jeffrey's prior leads to the improper gamma(1/2, 0) distribution
Since it's conjugate, we can still compute the proper posterior.
To condition on the date, we add the number of years to $\beta$ and the total number to $\alpha$.

In [49]:
prior_alpha = 1/2
prior_beta = 0
n_years = df.shape[0]
total_fatal = df.fatal.sum()
posterior_alpha = prior_alpha + total_fatal
posterior_beta = prior_beta + n_years

posterior = gamma(a=posterior_alpha, scale=1/posterior_beta)#fuck scipy

In [50]:
theta_samples = posterior.rvs(100000)
fatal_samples = poisson(theta_samples).rvs()
pred_interval = np.percentile(fatal_samples, [2.5, 97.5]).round(4)
print(pred_interval)

[14. 34.]


In [56]:
#13b
miles = 1e8 * (df.deaths / df.deathrate)
total_miles = miles.sum()
posterior_alpha = prior_alpha + total_fatal
posterior_beta = prior_beta + total_miles
posterior = gamma(a=posterior_alpha, scale=1/posterior_beta)
theta_samples = posterior.rvs(100000)
miles_1986 = 8e11
fatal_samples = poisson(miles_1986 * theta_samples).rvs()
pred_interval = np.percentile(fatal_samples, [2.5, 97.5]).round(4)
pred_interval

c) and d) are "completely analogous" (shoutout to Joe Blitzstein)